In this exercise, you will use your new knowledge to train a model with **gradient boosting**.

# Setup

The questions below will give you feedback on your work. Run the following cell to set up the feedback system.

In [1]:
# Set up code checking
import os
if not os.path.exists("../input/train.csv"):
    os.symlink("../input/home-data-for-ml-course/train.csv", "../input/train.csv")  
    os.symlink("../input/home-data-for-ml-course/test.csv", "../input/test.csv") 
import pandas as pd
import numpy as np

print("Setup Complete")

Setup Complete


You will work with the [Housing Prices Competition for Kaggle Learn Users](https://www.kaggle.com/c/home-data-for-ml-course) dataset from the previous exercise. 

![Ames Housing dataset image](https://i.imgur.com/lTJVG4e.png)

Run the next code cell without changes to load the training and validation sets in `X_train`, `X_valid`, `y_train`, and `y_valid`.  The test set is loaded in `X_test`.

In [2]:
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data

#X_train = X.drop(['Alley','PoolQC','MiscFeature','Fence',],axis=1)
#X_test_full = X_test_full.drop(['Alley','PoolQC','MiscFeature','Fence',],axis=1)
X_train=X.copy()

In [3]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [4]:
categorical_cols = [cname for cname in X_train.columns if
                    X_train[cname].nunique() < 20 and 
                    X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if 
                X_train[cname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train[my_cols].copy()
#X_valid = X_valid_full[my_cols].copy()
X_test = X_test_full[my_cols].copy()

In [5]:
X_train=pd.get_dummies(X_train[categorical_cols],drop_first=False)
X_test = pd.get_dummies(X_test[categorical_cols],drop_first=False)


In [6]:
X_train, X_test = X_train.align(X_test, join='left', axis=1)

In [7]:
X_train.head()


,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
3,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
4,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,1,0,0,0,0,0
5,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


In [8]:

X_test.head()

,MSZoning_C (all),MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,LotShape_IR1,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1461,0,0,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
1462,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1463,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1464,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0
1465,0,0,0,1,0,0,1,0,0,1,...,0,0,0,1,0,0,0,0,1,0


In [9]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='constant')
imputer = imputer.fit(X_train)
X_train = imputer.transform(X_train)



In [10]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_train, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [11]:

import xgboost as xgb

model = xgb.XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4,early_stopping_rounds=5)

model.fit(X_train_full, y_train, eval_set=[(X_valid_full, y_valid)])


[0]	validation_0-rmse:190291.29825
[1]	validation_0-rmse:181587.08165
[2]	validation_0-rmse:173319.62824
[3]	validation_0-rmse:165508.28536
[4]	validation_0-rmse:158122.18493
[5]	validation_0-rmse:151107.87248
[6]	validation_0-rmse:144530.21700
[7]	validation_0-rmse:138374.03418
[8]	validation_0-rmse:132490.51935
[9]	validation_0-rmse:126881.22948
[10]	validation_0-rmse:121654.91983
[11]	validation_0-rmse:116651.38691
[12]	validation_0-rmse:112082.42706
[13]	validation_0-rmse:107621.26897
[14]	validation_0-rmse:103540.32892
[15]	validation_0-rmse:99598.38702
[16]	validation_0-rmse:96034.43946
[17]	validation_0-rmse:92470.83407
[18]	validation_0-rmse:89048.76367
[19]	validation_0-rmse:85997.36597
[20]	validation_0-rmse:83129.54373
[21]	validation_0-rmse:80365.06842
[22]	validation_0-rmse:77851.95621
[23]	validation_0-rmse:75409.04448
[24]	validation_0-rmse:73255.29818
[25]	validation_0-rmse:71159.23110
[26]	validation_0-rmse:69258.38477
[27]	validation_0-rmse:67569.80932
[28]	validation

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=5, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=4, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [12]:
#model = xgb.XGBRegressor(n_estimators=133, learning_rate=0.05, n_jobs=4)
#model.fit(X_train_full, y_train)
preds_test= model.predict(X_test)

In [13]:
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)

# Keep going

Continue to learn about **[data leakage](https://www.kaggle.com/alexisbcook/data-leakage)**.  This is an important issue for a data scientist to understand, and it has the potential to ruin your models in subtle and dangerous ways!

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intermediate-machine-learning/discussion) to chat with other learners.*